In [7]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI

# Load database
db = SQLDatabase.from_uri("sqlite:///movies.db")
llm = ChatOpenAI(model="gpt-4o-mini")

# Create toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()


In [8]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example query
agent.run("Show me all movies after 2020 sorted by rating")




> Entering new AgentExecutor chain...
To answer the question, I need to find the relevant table that contains movie data, including the release year and rating. First, I will check the tables available in the database.

Action: sql_db_list_tables  
Action Input: ""  
Observation: movies
Thought:I found that there is a table named 'movies'. Next, I will check the schema of this table to understand its structure, particularly to find fields related to the release year and rating.

Action: sql_db_schema  
Action Input: "movies"  
Observation: 
CREATE TABLE movies (
	id INTEGER, 
	name TEXT, 
	release_date DATE, 
	genre TEXT, 
	description TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from movies table:
id	name	release_date	genre	description
1	Movie Journey 1	2013-02-16	Horror	A horror story about battle and love.
2	Movie Quest 2	1984-03-19	Fantasy	A fantasy story about dream and quest.
3	Movie Legend 3	2021-01-25	Drama	A drama story about warrior and shadow.
*/
Thought:The 'movies' table includ

'Retrieved movies released after 2020, sorted by release date.'

In [9]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.OPENAI_FUNCTIONS,  # instead of ZERO_SHOT_REACT_DESCRIPTION
    verbose=True
)
agent.run("Show me all movies after 2020 sorted by rating")




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


movies
Invoking: `sql_db_schema` with `{'table_names': 'movies'}`



CREATE TABLE movies (
	id INTEGER, 
	name TEXT, 
	release_date DATE, 
	genre TEXT, 
	description TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from movies table:
id	name	release_date	genre	description
1	Movie Journey 1	2013-02-16	Horror	A horror story about battle and love.
2	Movie Quest 2	1984-03-19	Fantasy	A fantasy story about dream and quest.
3	Movie Legend 3	2021-01-25	Drama	A drama story about warrior and shadow.
*/
Invoking: `sql_db_query_checker` with `{'query': "SELECT name, release_date FROM movies WHERE release_date > '2020-01-01' ORDER BY release_date DESC"}`


```sql
SELECT name, release_date FROM movies WHERE release_date > '2020-01-01' ORDER BY release_date DESC
```
Invoking: `sql_db_query` with `{'query': "SELECT name, release_date FROM movies WHERE release_date > '2020-01-01' ORDER BY release_date DESC"}`


[('Movie Legen

'Here are some movies released after 2020, sorted by their release date:\n\n1. Movie Legend 81 - Release Date: 2023-10-28\n2. Movie Shadow 56 - Release Date: 2023-10-21\n3. Movie Legend 52 - Release Date: 2023-02-25\n4. Movie Warrior 49 - Release Date: 2022-10-08\n5. Movie Warrior 26 - Release Date: 2022-04-23\n6. Movie Dream 71 - Release Date: 2022-02-21\n7. Movie Legend 74 - Release Date: 2021-06-01\n8. Movie Legend 50 - Release Date: 2021-04-21\n9. Movie Secret 45 - Release Date: 2021-04-05\n10. Movie Love 31 - Release Date: 2021-02-11\n\nLet me know if you need more information!'